In [19]:
import numpy as np
import pandas as pd

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(5, 128), # 첫 번째 선형 계층, 입력 차원은 5, 출력 차원은 128
            nn.BatchNorm1d(128), # 배치 정규화 계층, 출력 차원과 일치해야 함.
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128,256), # 두 번째 선형 계층, 입력 차원은 128, 출력 차원은 256
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(256, 128), # 세 번째 선형 계층
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, 1), # 네 번째 선형 계층
            nn.Sigmoid() # 출력값을 0과 1 사이로 압축하는 시그모이드 함수
        )
        
    def forward(self, x):
        x = x.view(x.size(0), -1) # 입력 텐서를 2차원 텐서로 reshape (view 메소드 사용,  여기서 x.size(0) = x의 첫번째 차원 크기 = batch size,   -1은 나머지 모든 차원을 일렬로 펼치겠다는 의미)
        x = self.classifier(x) # 아까 __init__에서 self.classifier (=모델) 정의했음.
        return x

In [23]:
# data_set 구하기
data_set = pd.concat((train.drop(['Survived'], axis = 1), test), axis = 0) # drop : 특정 label을 찾아서 해당 열(axis=1)을 삭제,    concat : 두 행렬(판다스 데이터프레임)을 이어붙임. 세로로, 행렬 A 마지막 행 밑에 행렬 B의 행을 (axis=0)
data_set = data_set.drop(['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis = 1) # 필요없는 열 삭제
data_set = data_set.fillna(data_set.mean()) # fillna : Not Available (=na) 값을 fill(채우다). 뭐로? 해당 열의 mean(평균)으로


# train_x, train_y, test_x 구하기
n_train = train.shape[0] # train set의 크기를 저장
train_x, test_x = data_set[:n_train], data_set[n_train:]
train_y = train['Survived'] # 'Survived' 열을 label(정답)으로 사용

train_x = train_x[train_x.keys()].values # keys : 제목 열의 모든 값 반환,    values : 판다스 데이터프레임을 numpy 배열로 변환
test_x = test_x[test_x.keys()].values
train_y = train_y.values # train_y는 어차피 열이 하나라서 keys 안 써도 됨.


# import
import torch.optim as optim
from torch.autograd import Variable


# 신경망 모델 설정하기
simple_nn = SimpleNN() # 초기화
optimizer = optim.Adam(simple_nn.parameters(), lr=0.01) # Adam 최적화 알고리즘 사용. 단순 경사하강법 대신.
error = nn.BCELoss() # 이진 교차 엔트로피 손실 함수(Binary Cross-Entropy Loss) 사용. --> 이진 분류 문제에서 모델의 예측 확률과 실제 라벨 간의 차이를 측정하여 손실을 계산하는 함수


# mini batch 사용
batch_size = 99
batch_count = int(len(train_x) / batch_size) # batch 개수


# 훈련 진행하기
for epoch in range(300): # 각 에폭에 대해
    train_loss = 0
    num_right = 0
    
    for i in range(batch_count): # 각 mini batch에 대해
        # 각 mini batch 구하기
        start = i * batch_size
        end = start + batch_size
        tensor_x = torch.FloatTensor(train_x[start:end]) # FloatTensor : 텐서로 변환
        tensor_y = torch.FloatTensor(train_y[start:end]).reshape(-1, 1) # reshape에서 -1: 크기를 알아서 계산하라
        
        # training
        optimizer.zero_grad() # 기울기를 zero로 초기화 --> 이전 단계의 역전파에서 계산한 기울기를 날리고 0에서 시작. 근데 우린 단순 경사하강법 대신 Adam 최적화 알고리즘 쓸 거니까 optimizer의 기울기를 초기화.
        output = simple_nn(tensor_x) # 앞에서 설정한 모델(simple_nn)을 사용하여 예측 수행
        loss = error(output, tensor_y) # 앞에서 설정한 error 함수 사용하여 loss 계산
        loss.backward() # 역전파 --> 기울기 계산
        optimizer.step() # Adam 최적화 알고리즘 사용해서 가중치 업뎃
        
        # 결과
        train_loss += loss.item() * batch_size # 모든 mini batch의 loss를 누적
        result = [1 if out >= 0.5 else 0 for out in output] # 모델을 통과시켜 나온 값이 0.5 넘느나 안 넘느나 갖고 예측 결과를 0 or 1로 반환
        num_right += np.sum(np.array(result) == train_y[start:end]) # 맞게 예측한 data 개수 누적

    # 이 에폭에서의 loss, accuracy 계산
    train_loss = train_loss / len(train_x)
    accuracy = num_right / len(train_x)
    
    # 25 에폭마다 진행상황 출력
    if epoch % 25 == 0:
        print('Loss: {} Accuracy: {}% Epoch:{}'.format(train_loss, accuracy, epoch))

        
# 종료
print('Training Ended')

Loss: 0.6510948406325446 Accuracy: 0.6588103254769921% Epoch:0
Loss: 0.5403163731098175 Accuracy: 0.7362514029180696% Epoch:25
Loss: 0.4824244413110945 Accuracy: 0.7811447811447811% Epoch:50
Loss: 0.4348728491200341 Accuracy: 0.8080808080808081% Epoch:75
Loss: 0.39422735902998185 Accuracy: 0.8305274971941639% Epoch:100
Loss: 0.3685516615708669 Accuracy: 0.8383838383838383% Epoch:125
Loss: 0.32133814361360336 Accuracy: 0.8462401795735129% Epoch:150
Loss: 0.30069151851865983 Accuracy: 0.8641975308641975% Epoch:175
Loss: 0.31094634532928467 Accuracy: 0.8653198653198653% Epoch:200
Loss: 0.258775500787629 Accuracy: 0.8911335578002245% Epoch:225
Loss: 0.2697751290268368 Accuracy: 0.8978675645342312% Epoch:250
Loss: 0.2251274800962872 Accuracy: 0.9090909090909091% Epoch:275
Training Ended


In [24]:
tensor_test_x = torch.FloatTensor(test_x) # test_x를 텐서로 바꾸기

with torch.no_grad(): # 이 블록 내 연산은 기록되지 않고, 기울기 업데이트에 영향을 주지 X
    # test 수행
    test_output = simple_nn(tensor_test_x) # test_x를 모델에 통과시켜서 output 얻기
    result = np.array([1 if out >= 0.5 else 0 for out in test_output]) # output >= 0.5인지 보고 result = 0 or 1 반환
    
    # test 결과를 csv 파일에 기록
    submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': result}) # 결과를 CSV 파일로 저장하기 위해 데이터프레임을 생성
    submission.to_csv('submission.csv', index=False) # CSV 파일로 저장